# Code to explore data a bit

### Check the number of types :

In [43]:
import pandas as pd

def count_classes(data_file):
    """
    Counts the number of unique classes (types) in the dataset.

    Args:
        data_file (str): Path to the dataset file (CSV format).

    Returns:
        dict: A dictionary with class types as keys and their counts as values.
    """
    # Load the dataset
    data = pd.read_csv(data_file)

    # Count the occurrences of each class
    class_counts = data["type"].value_counts().to_dict()

    return class_counts

# Example usage
data_file = "data/data.csv"  # Replace with your dataset's file path
class_counts = count_classes(data_file)
print("Class Counts:", class_counts)

Class Counts: {'water': 115, 'normal': 107, 'bug': 91, 'grass': 71, 'psychic': 56, 'fire': 51, 'electric': 49, 'rock': 44, 'dragon': 32, 'ground': 32, 'dark': 31, 'steel': 28, 'poison': 28, 'fighting': 27, 'ghost': 26, 'ice': 24, 'fairy': 17, 'flying': 4}


In [44]:
print("Number of types:",len(class_counts))

Number of types: 18


### Check the size of images:

In [45]:
import os
from PIL import Image
import pandas as pd

def count_image_sizes(img_dir, data_file):
    """
    Counts the sizes of all images in the dataset.

    Args:
        img_dir (str): Directory where images are stored.
        data_file (str): Path to the CSV file containing image data.

    Returns:
        dict: A dictionary with image sizes as keys and their counts as values.
    """
    # Load the dataset
    data = pd.read_csv(data_file)

    # Dictionary to store size counts
    size_counts = {}

    for img_name in data["image"]:
        img_path = os.path.join(img_dir, img_name)
        if os.path.exists(img_path):
            with Image.open(img_path) as img:
                size = img.size  # (width, height)
                if size not in size_counts:
                    size_counts[size] = 1
                else:
                    size_counts[size] += 1

    return size_counts

# Example usage
img_dir = "data/images"  # Replace with the actual image directory path
data_file = "data/data.csv"  # Replace with the actual dataset file path
sizes = count_image_sizes(img_dir, data_file)
print("Image Sizes and Counts:", sizes)


Image Sizes and Counts: {(1280, 1280): 453, (431, 431): 258, (854, 854): 1, (1080, 1080): 27, (272, 272): 12, (405, 405): 1, (789, 789): 1, (1279, 1279): 1, (859, 859): 2, (899, 899): 3, (600, 600): 1, (944, 944): 1, (1029, 1029): 1, (399, 399): 1, (893, 893): 1, (950, 950): 1, (892, 892): 1, (937, 937): 4, (1043, 1043): 2, (1022, 1022): 1, (915, 915): 2, (864, 864): 1, (1127, 1127): 1, (850, 850): 1, (980, 980): 1, (931, 931): 1, (886, 886): 1, (664, 664): 1, (1261, 1261): 1, (1162, 1162): 1, (429, 429): 1, (860, 860): 1, (872, 872): 1, (766, 766): 1, (426, 426): 1, (834, 834): 2, (1137, 1137): 1, (957, 957): 1, (857, 857): 1, (1246, 1246): 1, (755, 755): 1, (862, 862): 3, (414, 414): 1, (1021, 1021): 1, (516, 516): 1, (594, 594): 1, (889, 889): 1, (418, 418): 1, (1032, 1032): 2, (956, 956): 1, (680, 680): 1, (878, 878): 1, (845, 845): 1, (856, 856): 1, (868, 868): 1, (1024, 1024): 1, (839, 839): 1, (869, 869): 2, (417, 417): 1, (410, 410): 1, (401, 401): 1, (970, 970): 1, (1125, 1125

### We will transform to the smallest

In [46]:
def find_smallest_image_size(img_dir, data_file):
    """
    Finds the smallest image size in the dataset.

    Args:
        img_dir (str): Directory where images are stored.
        data_file (str): Path to the CSV file containing image data.

    Returns:
        tuple: The dimensions of the smallest image (width, height).
    """
    # Load the dataset
    data = pd.read_csv(data_file)

    # Initialize minimum size
    min_size = None

    for img_name in data["image"]:
        img_path = os.path.join(img_dir, img_name)
        if os.path.exists(img_path):
            with Image.open(img_path) as img:
                size = img.size  # (width, height)
                if min_size is None or (size[0] * size[1] < min_size[0] * min_size[1]):
                    min_size = size

    return min_size

smallest_size = find_smallest_image_size(img_dir, data_file)
smallest_size

(272, 272)

### Visualisation des prédictions du modèle CNN

In [47]:
import torch
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
from cnn import FlexibleCNN
from dataset import CustomPKMNDataset
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

def load_model(model_path, num_classes, device="cpu"):
    """
    Load the trained model from a .pth file.
    """
    model = FlexibleCNN(input_channels=3, num_classes=num_classes)
    try:
        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict)
        model.to(device)
        model.eval()
        print(f"Model loaded successfully from {model_path}.")
        return model
    except RuntimeError as e:
        print(f"Error loading model: {e}")
        raise

def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-length sequences.
    Returns:
        images: Tensor of shape [batch_size, channels, height, width]
        captions: LongTensor of shape [batch_size, max_seq_length]
        type_indices: LongTensor of shape [batch_size]
    """
    images = torch.stack([item[0] for item in batch])  # Stack all images
    captions = [item[1] for item in batch]             # List of caption tensors
    padded_captions = pad_sequence(captions, batch_first=True, padding_value=0)
    type_indices = torch.tensor([item[2] for item in batch], dtype=torch.long)
    return images, padded_captions, type_indices

def show_predictions(model, dataloader, class_names, device="cpu"):
    """
    Show a grid of images with their actual and predicted classes.
    """
    model.eval()
    try:
        # Get a single batch: images, captions, and type_indices
        images, _, type_indices = next(iter(dataloader))
        images, labels = images.to(device), type_indices.to(device)

        # Get model predictions
        with torch.no_grad():
            outputs = model(images)
            probabilities = softmax(outputs, dim=1)
            predictions = torch.argmax(probabilities, dim=1)

        # Convert images to PIL format for plotting
        images = images.cpu()
        images = [ToPILImage()(img) for img in images]

        # Plot 16 images in a 4x4 grid
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        fig.suptitle("Actual vs Predicted Classes", fontsize=16)

        for idx, ax in enumerate(axes.flat):
            if idx < len(images):
                ax.imshow(images[idx])
                ax.axis("off")
                actual_class = class_names[labels[idx].item()]
                predicted_class = class_names[predictions[idx].item()]
                ax.set_title(f"A: {actual_class}\nP: {predicted_class}")
            else:
                ax.axis("off")

        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        plt.show()
    except Exception as e:
        print(f"Error during prediction: {e}")
        raise

# Parameters
img_dir = "data/images"
data_file = "data/data.csv"
batch_size = 16
model_path = "best_model_cnn.pth"

# Use the exact same class_names as used during training
class_names = [
    "normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground",
    "flying", "psychic", "bug", "rock", "ghost", "dragon", "dark", "steel", "fairy"
]

num_classes = len(class_names)

# Transformations
transform = transforms.Compose([
    transforms.Resize((272, 272)),
    transforms.ToTensor()
])

# Dataset and DataLoader
try:
    dataset = CustomPKMNDataset(
        img_dir, 
        data_file, 
        transform=transform, 
        class_to_idx={cls_name: i for i, cls_name in enumerate(class_names)}
    )
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
    print("Dataset and DataLoader initialized successfully.")
except Exception as e:
    print(f"Error initializing dataset or dataloader: {e}")
    raise

# Load model
try:
    model = load_model(model_path, num_classes, device="cuda" if torch.cuda.is_available() else "cpu")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Show predictions
try:
    show_predictions(model, dataloader, class_names, device="cuda" if torch.cuda.is_available() else "cpu")
except Exception as e:
    print(f"Error displaying predictions: {e}")
    raise


Error initializing dataset or dataloader: __init__() got an unexpected keyword argument 'class_to_idx'


TypeError: __init__() got an unexpected keyword argument 'class_to_idx'

### Same for CNN with attention

In [ ]:
from cnn_with_attention import FlexibleCNNWithAttention

def load_model(model_path, num_classes, device="cpu"):
    """
    Load the trained model from a .pth file.
    """
    model = FlexibleCNNWithAttention(input_channels=3, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def show_predictions(model, dataloader, class_names, device="cpu"):
    """
    Show a grid of images with their actual and predicted classes.
    """
    model.eval()
    images, labels, _ = next(iter(dataloader))  # Get a single batch
    images, labels = images.to(device), labels[:, 0].to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(images)
        probabilities = softmax(outputs, dim=1)
        predictions = torch.argmax(probabilities, dim=1)

    # Convert images to PIL format for plotting
    images = images.cpu()
    images = [ToPILImage()(img) for img in images]

    # Plot 16 images in a 4x4 grid
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    fig.suptitle("Actual vs Predicted Classes", fontsize=16)

    for idx, ax in enumerate(axes.flat):
        if idx < len(images):
            ax.imshow(images[idx])
            ax.axis("off")
            actual_class = class_names[labels[idx].item()]
            predicted_class = class_names[predictions[idx].item()]
            ax.set_title(f"A: {actual_class}\nP: {predicted_class}")
        else:
            ax.axis("off")

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

# Parameters
img_dir = "data/images"
data_file = "data/data.csv"
batch_size = 16
model_path = "best_model_cnn_with_attention.pth"

# Transformations
transform = transforms.Compose([
    transforms.Resize((272, 272)),
    transforms.ToTensor()
])

# Dataset and DataLoader
dataset = CustomPKMNDataset(img_dir, data_file, transform=transform)
class_names = dataset.vocab.itos  # Assuming vocab has an `itos` method for class names
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load model
num_classes = len(class_names) - 4  # Adjust for padding or special tokens
model = load_model(model_path, num_classes, device="cuda" if torch.cuda.is_available() else "cpu")

# Show predictions
show_predictions(model, dataloader, class_names, device="cuda" if torch.cuda.is_available() else "cpu")

C:\Users\emac\AppData\Local\Temp\ipykernel_11644\1189899858.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

FileNotFoundError: [Errno 2] No such file or directory: 'best_model_cnn_with_attention.pth'